# 5. 차원 축소를 사용한 데이터 압축

차원 축소를 위한 특성 선택의 또 다른 방식은 특성 추출(feature extraction)이다  
이 장에서는 데이터셋의 정보를 요약하는 세 가지 기본적인 기술을 배울 것이다  
이들은 원본 데이터셋을 좀 더 낮은 차원의 새로운 특성 부분 공간으로 변환한다  

* 주성분 분석(Principal Component Analysis, PCA)을 사용한 비지도(unsupervised) 데이터 압축하기
* 지도(supervised) 방식의 차원 축소 기법인 선형 판별 분석(Linear Discriminant Analysis, LDA)을 이용하여 클래스 구별 능력 최대화하기
* 커널 PCA(kernel Principal Component Analysis, KPCA)를 사용한 비선형 차원 축소하기  

## 5.1 주성분 분석을 통한 비지도 차원 축소

특성 선택과 특성 추출의 차이는 원본 특성을 유지하느냐에 있다  
순차 후진 선택같은 특성 선택 알고리즘을 사용할 때는 원본 특성을 유지하지만 특성 추출은 새로운 특성 공간으로 데이터를 변환하거나 투영한다  
차원 축소 관점에서 보면 특성 추출은 대부분의 관련있는 정보를 유지하면서 데이터를 압축하는 방법으로 이해할 수 있다  
특성 추출이 저장 공간을 절약하거나 학습 알고리즘의 계산 효율성을 향상할 뿐만 아니라 차원의 저주 문제를 감소시켜 예측 성능을 향상하기도 한다  

### 5.1.1 주성분 분석의 주요 단계

PCA는 특성 사이의 상관관계를 기반으로 하여 데이터에 있는 특성을 잡아낼 수 있다  
PCA는 고차원 데이터에서 분산이 가장 큰 방향을 찾고 좀 더 작거나 같은 수의 차원을 갖는 새로운 부분 공간으로 이를 투영합니다  
새로운 부분 공간의 직교 좌표는 주오진 조건하에서 분산이 최대인 방향으로 해석할 수 있다  
<img src='https://blog.kakaocdn.net/dn/bHVVHh/btqBVgfijCd/Kkkehx2uyeKtFpSdQAOjz1/img.png'>
$x_{1}$과 $x_{2}$는 원본 특성 축이고 PC1과 PC2는 주성분이다  

PCA를 사용하여 차원을 축소하기 위해 $d$x$k$차원의 변환 행렬 $W$를 만든다  
이 행렬로 샘플 벡터 $x$를 새로운 $k$차원의 특성 부분 공간으로 매핑한다  
이 부분 공간은 원본 $d$차원의 특성 공간보다 작은 차원을 갖는다  
$$
x = \left[x_{1},x_{2},\cdots,x_{d}\right], x\in\mathbb{R}^{d}\\
\downarrow xW, W\in\mathbb{R}^{d\times k}\\
z=\left[z_{1},z_{2},\cdots,z_{k}\right], z\in \mathbb{R}^{k}
$$
원본 $d$ 차원 데이터를 새로운 $k$ 차원의 부분 공간으로 변환하여 만들어진 첫 번째 주성분이 가장 큰 분산을 가진다  
모든 주성분들과 상관관계가 없다는 제약하에 가장 큰 분산을 가진다  
입력 특성에 상관관계가 있더라도 만들어진 주성분은 서로 직각을 이룰것이다  
PCA방향은 데이터 스케일에 매우 민감하다  
특성의 스케일이 다르고 모든 특성의 중요도를 동일하게 취급하려면 PCA를 적용하기 전에 특성을 표준화 전처리해야한다  

차원 축소를 위한PCA 알고리즘을 자세히 알아보기 전에 사용할 방법을 몇 단계로 나누어 정리하면
1. $d$차원 데이터셋을 표준화 전처리한다  
2. 공분산 행렬을 만든다  
3. 공분산 행렬을 고유 벡터와 고윳값으로 분해한다  
4. 고윳값을 내림차순으로 정렬하고 그에 해당하는 고유 벡터의 순위를 매긴다  
5. 고윳값이 가장 큰 $k$개의 고유 벡터를 선택한다. 여기서 $k$는 새로운 특성 부분 공간의 차원이다($k\leq d$)  
6. 최상위 $k$개의 고유 벡터로 투영 행렬 $W$를 만든다  
7. 투영 행렬 $W$를 사용해서 $d$ 차원 입력 데이터셋 $X$를 새로운 $k$ 차원의 특성 부분 공간으로 변환한다  

### 5.1.2 주성분 추출 단계

이 절에서 PCA 처음 네 단계를 처리한다  
1. 데이터를 표준화 전처리한다  
2. 공분산 행렬을 구성한다  
3. 공분산 행렬의 고윳값과 고유 벡터를 구한다  
4. 고윳값을 내림차순으로 정렬하여 고유 벡터의 순위를 매긴다  

Wine 데이터셋 로드

In [8]:
import pandas as pd
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/'
                      'machine-learning-databases/wine/wine.data',
                       header=None)

Wine 데이터셋을 70%와 30% 비율로 훈련 세트와 테스트 세트로 나누고 표준화를 적용하여 단위 분산을 갖도록 한다

In [9]:
from sklearn.model_selection import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3,
                     stratify=y,
                     random_state=0)
# 특성을 표준화 전처리한다  
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

앞 코드를 실행하여 필수적인 전처리 단계를 완료한 후 공분산 행렬을 만드느 두 번째 단계를 진행한다  
공분산 $d \times d$차원의 대칭 행렬로 특성 상호간의 공분산을 저장한다  
$d$는 데이터셋에 있는 차원 개수입니다  
예를 들어 전체 샘플에 대한 두 특성 $x_{j}$와 $x_{k}$ 사이의 공분산은 다음 식으로 계산 할 수 있다  
$$
\sigma_{jk} = \frac{1}{n}\sum^{n}_{i=1}\left(x_{j}^{\left(i\right)} - \mu_{j}\right)\left(x_{k}^{\left(i\right)} - \mu_{k}\right)
$$
여기서 $\mu_{j}$와 $\mu_{k}$는 특성 $j$와 $k$의 샘플 평균이다  
데이터셋을 표준화 전처리했기 때문에 샘플 평균은 0이다  
두 특성 간 양의 공분산은 특성이 함께 증가하거나 감소하는 것을 나타낸다  
반면 음의 공분산은 특성이 반대 방향으로 달라진다는 것을 나타낸다  
예를 들어 세 개의 특성으로 이루어진 공분산 행렬은 다음과 같이 쓸 수 있다  
$$
\sum = \begin{bmatrix}
\sigma_{1}^{2} & \sigma_{12} & \sigma_{13}\\ 
\sigma_{21} & \sigma_{2}^{2} & \sigma_{23}\\ 
\sigma_{31} & \sigma_{32} & \sigma_{3}^{2}
\end{bmatrix}
$$
공분산 행렬의 고유 벡터가 주성분(최대 분산의 방향)을 표현합니다  
이에 대응되는 고윳값은 주성분의 크기이다  
Wine 데이터셋의 경우 13 X 13 차원의 공분산 행렬로부터 13개의 소유 벡터와 고윳값을 얻을 수 있다  

이제 세 번째 단계를 위해 공분사의 행렬의 고유 벡터와 고윳값의 쌍을 구해보자  
고유 벡터 $v$는 다음 식을 만족한다  
$$
\sum v = \lambda v
$$
여기서 $\lambda$는 스케일을 담당하는 고윳값이다  
고윳값과 벡터를 직접 게산하는 것은 복잡하기 때문에 numpy의 linalg.eig 함수를 사용하여 Wine 데이터셋의 공분산 행렬에 대한 고유 벡터와 고윳값 쌍을 계산한다  
